In [1]:
import torch
import torch.nn as nn
import pandas as pd
from HGP.models import LightModel
import pickle

In [2]:
import networkx as nx

In [3]:
from utils.imports import *
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import SimpleProfiler

# BITTE EINMAL DEN MODELL NAMEN HIER EINMAL EINTRAGEN (HGP-SL o.ä.)

Damit die Datei unter dem Namen gespeichert wird

In [ ]:
model_name = "HGP"

In [6]:
# %%
args_dict = take_hp("configs/hpt-nci1001.yml")

pl.seed_everything(args_dict["random_seed"], workers=True)


Global seed set to 42


42

In [7]:
# %%
# Load the dataset
dataset = TUDataset(
    Path.cwd().joinpath("datasets"),
    name=args_dict["dataset"],
    use_node_attr=True,
)
args_dict["num_classes"] = dataset.num_classes
args_dict["num_features"] = dataset.num_features

Extracting /home/bjoern/AML22-project/datasets/NCI1/NCI1.zip
Processing...
Done!


In [8]:
split_train = int(dataset.len() * args_dict["split_ratio"])
split_val = int((dataset.len() - split_train) * args_dict["test_ratio"])
split_test = dataset.len() - split_val - split_train

print(
    f"Hole: {dataset.len()}, Train: {split_train}, Val: {split_val}, Test: {split_test}"
)
# %%
train_data, eval_data, test_data = random_split(
    dataset,
    [split_train, split_val, split_test],
    generator=torch.Generator().manual_seed(42),
)

Hole: 4110, Train: 2877, Val: 616, Test: 617


train_data, eval_data, test_data = random_split(
    dataset,
    [split_train, split_val, split_test],
    generator=torch.Generator().manual_seed(42),
)

In [9]:
# %%
train_loader = DataLoader(
    train_data,
    batch_size=args_dict["batch_size"],
    shuffle=False,
    num_workers=args_dict["num_workers"],
)
val_loader = DataLoader(
    eval_data,
    batch_size=args_dict["batch_size"],
    shuffle=False,
    num_workers=args_dict["num_workers"],
)
test_loader = DataLoader(
    test_data,
    batch_size= 1 ,# dataset.len() - split_test,
    shuffle=False,
    num_workers=args_dict["num_workers"],
)

In [12]:
# Define the model
model = LightModel(args_dict).to(args_dict["device"])
# load a saved model
weights = torch.load("model/hpt-nci1/hpt-nci1/model.pt")
# set the weights
model.load_state_dict(weights)

model.eval()

loss = nn.CrossEntropyLoss(reduction="none")
softmax = nn.Softmax(dim=1)

In [34]:

right_y = []
wrong_y = []

right_graphs = []
wrong_graphs = []

for batch in test_loader:
    data = batch.to(args_dict["device"])
    out = model(data)
    prediction_truth = np.argmax(softmax(out).detach().cpu().numpy()) == data.y.detach().cpu().numpy()
    
    if prediction_truth[0]:
        g = nx.from_edgelist(data.edge_index.cpu().numpy().T)
        right_graphs.append(g)
        right_y.append(data.y.detach().cpu().numpy()[0])
    else:
        g = nx.from_edgelist(data.edge_index.cpu().numpy().T)
        wrong_graphs.append(g)
        wrong_y.append(data.y.detach().cpu().numpy()[0])

np_right_graphs = np.array(right_graphs)
np_wrong_graphs = np.array(wrong_graphs)
np_right_y = np.array(right_y)
np_wrong_y = np.array(wrong_y)

In [ ]:
# save numpy arrays to folder analysis
np.save(f"analysis/{args_dict['dataset']}_{model_name}_right_graphs.npy", np_right_graphs)
np.save(f"analysis/{args_dict['dataset']}_{model_name}_wrong_graphs.npy", np_wrong_graphs)
np.save(f"analysis/{args_dict['dataset']}_{model_name}_right_y.npy", np_right_y)
np.save(f"analysis/{args_dict['dataset']}_{model_name}_wrong_y.npy", np_wrong_y)